In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
dfTrain = pd.read_csv(r"UNSW_2018_IoT_Botnet_V6_Train.csv", sep=",", low_memory=False)
dfTest = pd.read_csv(r"UNSW_2018_IoT_Botnet_V6_Test.csv", sep=",", low_memory=False)

### Reduction to 3 class

In [3]:
dfTrain['class'] = dfTrain['class'].replace(['DoS-TCP'], 'Attack-TCP')
dfTrain['class'] = dfTrain['class'].replace(['DoS-UDP'], 'Attack-UDP')
dfTrain['class'] = dfTrain['class'].replace(['DDoS-TCP'], 'Attack-TCP')
dfTrain['class'] = dfTrain['class'].replace(['DDoS-UDP'], 'Attack-UDP')
dfTrain['class'] = dfTrain['class'].replace(['DDoS-HTTP'], 'Attack-TCP')
dfTrain['class'] = dfTrain['class'].replace(['DoS-HTTP'], 'Attack-TCP')
dfTrain['class'] = dfTrain['class'].replace(['Normal'], 'Normal')

In [4]:
dfTest['class'] = dfTest['class'].replace(['DoS-TCP'], 'Attack-TCP')
dfTest['class'] = dfTest['class'].replace(['DoS-UDP'], 'Attack-UDP')
dfTest['class'] = dfTest['class'].replace(['DDoS-TCP'], 'Attack-TCP')
dfTest['class'] = dfTest['class'].replace(['DDoS-UDP'], 'Attack-UDP')
dfTest['class'] = dfTest['class'].replace(['DDoS-HTTP'], 'Attack-TCP')
dfTest['class'] = dfTest['class'].replace(['DoS-HTTP'], 'Attack-TCP')
dfTest['class'] = dfTest['class'].replace(['Normal'], 'Normal')

### Dataset Information 

In [5]:
dfTrain.sample(10)

,pkts,bytes,dur,spkts,dpkts,sbytes,dbytes,rate,srate,drate,class
3058157,13,780,14.512684,13,0,780,0,0.826863,0.826863,0.000000,Attack-UDP
2331912,2,132,0.106100,2,0,132,0,9.425071,9.425071,0.000000,Normal
246103,11,660,13.799225,11,0,660,0,0.724678,0.724678,0.000000,Attack-UDP
236882,7,890,32.062889,5,2,770,120,0.187132,0.124887,0.038438,Attack-TCP
1131403,10,600,15.038621,10,0,600,0,0.598459,0.598459,0.000000,Attack-UDP
2053086,9,540,26.274174,9,0,540,0,0.304482,0.304482,0.000000,Attack-UDP
1151583,5,676,56.414722,4,1,616,60,0.070903,0.053178,0.000000,Attack-TCP
1313718,7,420,25.332344,7,0,420,0,0.236851,0.236851,0.000000,Attack-UDP
1156292,4,428,9.875354,3,1,368,60,0.303787,0.202524,0.000000,Attack-TCP
1103201,5,770,15.900399,5,0,770,0,0.251566,0.251566,0.000000,Attack-TCP


In [6]:
dfTest.sample(10)

,pkts,bytes,dur,spkts,dpkts,sbytes,dbytes,rate,srate,drate,class
1334248,7,420,31.897711,7,0,420,0,0.188101,0.188101,0.000000,Attack-UDP
1274337,6,360,11.060322,6,0,360,0,0.452066,0.452066,0.000000,Attack-UDP
490563,2,482,0.302959,1,1,92,390,3.300777,0.000000,0.000000,Normal
772960,6,830,28.731071,5,1,770,60,0.174028,0.139222,0.000000,Attack-TCP
79568,15,900,13.965811,15,0,900,0,1.002448,1.002448,0.000000,Attack-UDP
330223,1,83,0.000000,1,0,83,0,0.000000,0.000000,0.000000,Normal
1062810,6,548,23.478559,4,2,428,120,0.212960,0.127776,0.042592,Attack-TCP
151610,13,780,14.338960,13,0,780,0,0.836881,0.836881,0.000000,Attack-UDP
616994,6,736,11.170774,5,1,676,60,0.447597,0.358077,0.000000,Attack-TCP
515794,4,616,23.426392,4,0,616,0,0.128061,0.128061,0.000000,Attack-TCP


### Train Dataset 

In [7]:
X_train = dfTrain.iloc[:,:-1]
y_train = dfTrain.iloc[:, -1].values
print(Counter(y_train))

Counter({'Attack-UDP': 1387413, 'Attack-TCP': 1117047, 'Normal': 722441})


In [8]:
X_test = dfTest.iloc[:,:-1]
y_test = dfTest.iloc[:, -1].values
print(Counter(y_test))

Counter({'Attack-UDP': 593817, 'Attack-TCP': 478607, 'Normal': 310534})


In [9]:
%%time
model = KNeighborsClassifier(n_neighbors=7)
model.fit(X_train, y_train)

Wall time: 46.1 s


KNeighborsClassifier(n_neighbors=7)

### Model Summary 

In [10]:
y_pred = model.predict(X_test)

In [11]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=4))

[[478602      1      4]
 [     9 593805      3]
 [     0      0 310534]]
              precision    recall  f1-score   support

  Attack-TCP     1.0000    1.0000    1.0000    478607
  Attack-UDP     1.0000    1.0000    1.0000    593817
      Normal     1.0000    1.0000    1.0000    310534

    accuracy                         1.0000   1382958
   macro avg     1.0000    1.0000    1.0000   1382958
weighted avg     1.0000    1.0000    1.0000   1382958



### Classification Prediction Validation   

In [12]:
a = 0
b = 0
for x in range(0, 921477):
    if (y_pred[x] == y_test[x]):
        a += 1
    else:
        b += 1
print('Classification Data')
print('Accurate :',a, 'Loss :',b)

Classification Data
Accurate : 921465 Loss : 12


In [13]:
Attack_UDP = 0
Attack_TCP = 0
Normal = 0
Attack_UDP_Loss = 0
Attack_TCP_Loss = 0
Normal_Loss = 0
success=0
loss = 0

for x in range(0, len(y_pred)):
    if (y_pred[x] == y_test[x]):
        success += 1
        if (y_pred[x] == 'Attack-UDP'):
            Attack_UDP += 1
        elif (y_pred[x] == 'Attack-TCP'):
            Attack_TCP += 1
        elif (y_pred[x] == 'Normal'):
            Normal += 1
    elif (y_pred[x] != y_test[x]):
        loss += 1
        if (y_test[x] == 'Attack-UDP'):
            Attack_UDP_Loss += 1
        elif (y_test[x] == 'Attack-TCP'):
            Attack_TCP_Loss += 1
        elif (y_test[x] == 'Normal'):
            Normal_Loss += 1
print('Classification Data')
print('Attack-UDP :',Attack_UDP, 'Attack-TCP :',Attack_TCP, 'Normal :',Normal)
print('Attack-UDP-Loss :',Attack_UDP_Loss, 'Attack-TCP-Loss :',Attack_TCP_Loss,'Normal-Loss :',Normal_Loss)
print('Success :', success, 'Loss :', loss)

Classification Data
Attack-UDP : 593805 Attack-TCP : 478602 Normal : 310534
Attack-UDP-Loss : 12 Attack-TCP-Loss : 5 Normal-Loss : 0
Success : 1382941 Loss : 17


### Simulation Real Traffic Data 

In [14]:
dfRealTCP = pd.read_csv(r"test-hping-v6-tcp-final.csv", sep=",", low_memory=False)
dfRealUDP = pd.read_csv(r"test-hping-v6-udp-final.csv", sep=",", low_memory=False)
dfRealNormal = pd.read_csv(r"test-v6-normal-final.csv", sep=",", low_memory=False)

In [15]:
dfRealTCP['Class'] = 'Attack-TCP'
dfRealUDP['Class'] = 'Attack-UDP'
dfRealNormal['Class'] = 'Normal'

In [16]:
dfReal = pd.concat([dfRealTCP,dfRealUDP,dfRealNormal], axis=0, join='outer', ignore_index=False)

In [17]:
dfReal.sample(20)

,SrcAddr,Sport,DstAddr,Dport,TotPkts,TotBytes,Dur,SrcPkts,DstPkts,SrcBytes,DstBytes,Rate,SrcRate,DstRate,Class
1283,192.168.1.3,5978,209.97.172.191,http,1,154,0.000000,1,0,154,0,0.000000,0.000000,0.000000,Attack-TCP
908,192.168.1.3,8068,209.97.172.191,80,1,1042,0.000000,1,0,1042,0,0.000000,0.000000,0.000000,Attack-UDP
124,192.168.1.3,5890,209.97.172.191,http,3,268,0.022286,2,1,208,60,89.742439,44.871220,0.000000,Attack-TCP
275,192.168.1.3,9436,209.97.172.191,80,1,1042,0.000000,1,0,1042,0,0.000000,0.000000,0.000000,Attack-UDP
1360,192.168.1.3,4443,209.97.172.191,http,3,268,0.021037,2,1,208,60,95.070595,47.535297,0.000000,Attack-TCP
1331,192.168.1.3,7992,209.97.172.191,80,1,1042,0.000000,1,0,1042,0,0.000000,0.000000,0.000000,Attack-UDP
2273,192.168.1.3,5507,209.97.172.191,80,1,1042,0.000000,1,0,1042,0,0.000000,0.000000,0.000000,Attack-UDP
597,192.168.1.3,3985,209.97.172.191,http,3,268,0.018412,2,1,208,60,108.624809,54.312405,0.000000,Attack-TCP
2594,192.168.1.3,3879,209.97.172.191,http,1,154,0.000000,1,0,154,0,0.000000,0.000000,0.000000,Attack-TCP
2892,192.168.1.3,3778,209.97.172.191,http,3,268,0.020405,2,1,208,60,98.015190,49.007595,0.000000,Attack-TCP


In [18]:
dfReal.shape

(6532, 15)

In [19]:
indentity = dfReal.iloc[:,:4]
label = dfReal.iloc[:, -1].values
features = dfReal.iloc[:,4:14].values
prediction = model.predict(features)
result = indentity
result['prediction'] = prediction

C:\Users\Duck~\.conda\envs\Tensorflow RNN\lib\site-packages\sklearn\base.py:446: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  "X does not have valid feature names, but"


In [20]:
print(Counter(label))

Counter({'Attack-TCP': 3000, 'Attack-UDP': 3000, 'Normal': 532})


In [21]:
print(confusion_matrix(label, prediction))
print(classification_report(label, prediction, digits=4))

[[3000    0    0]
 [   0 3000    0]
 [  32   49  451]]
              precision    recall  f1-score   support

  Attack-TCP     0.9894    1.0000    0.9947      3000
  Attack-UDP     0.9839    1.0000    0.9919      3000
      Normal     1.0000    0.8477    0.9176       532

    accuracy                         0.9876      6532
   macro avg     0.9911    0.9492    0.9681      6532
weighted avg     0.9878    0.9876    0.9871      6532



In [22]:
grouped_multi = result.groupby(['SrcAddr', 'DstAddr', 'Dport', 'prediction']).agg(prediction_count=('prediction', 'count')).reset_index(level=[0,1,2,3])

In [23]:
grouped_multi

,SrcAddr,DstAddr,Dport,prediction,prediction_count
0,192.168.1.2,209.97.172.191,https,Attack-TCP,32
1,192.168.1.2,209.97.172.191,https,Attack-UDP,49
2,192.168.1.2,209.97.172.191,https,Normal,451
3,192.168.1.3,209.97.172.191,80,Attack-UDP,3000
4,192.168.1.3,209.97.172.191,http,Attack-TCP,3000


### Dump Model to .pkl 

In [ ]:
import pickle
import pandas as pd

In [ ]:
pickle.dump(model, open('./FinalNBImblearnV6_3Class.pkl','wb'))